# Main Code - Hand Gesture Recognition Using OpenCV and MediaPipe

In [1]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import screen_brightness_control as sbc
import os
import vlc
import threading
import time

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

class MediaPlayerThread(threading.Thread):
    def __init__(self, file_paths):
        super().__init__()
        self.file_paths = file_paths
        self.current_index = 0
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
    
    def play(self):
        self.player.play()
    
    def pause(self):
        self.player.pause()
        
    def stop(self):
        self.player.stop()
    
    def next_song(self):
        self.current_index = (self.current_index + 1) % len(self.file_paths)
        self.player.stop()
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
    
    def previous_song(self):
        self.current_index = (self.current_index - 1) % len(self.file_paths)
        self.player.stop()
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
    
    def run(self):
        self.play()
        while True:
            time.sleep(1)  # Adjust the sleep time as needed

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Initialize audio devices
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

# Get volume range
volMin, volMax = volume.GetVolumeRange()[:2]

# Flag to track if brightness changed
brightness_changed = False

# Initialize media player thread
path = "media/"
file_names = os.listdir(path)
file_paths = [os.path.join(path, file) for file in file_names]
media_player_thread = MediaPlayerThread(file_paths)
media_player_thread.start()

# Adding a short delay to allow the media player to initialize
time.sleep(1)

# Main loop to capture frames from webcam
while True:
    # Capture frame from webcam
    success, img = cap.read()
    
    # Flip the frame horizontally for natural hand movements
    img = cv2.flip(img, 1)
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList, right_lmList = [], []
    
    # Check if hands are detected and determine their labels (left or right)
    if results.multi_hand_landmarks and results.multi_handedness:
        for i in results.multi_handedness:
            label = i.classification[0].label
            if label == 'Left':
                # Extract and store landmark positions for left hand
                for lm in results.multi_hand_landmarks[0].landmark:
                    h, w, _ = img.shape
                    left_lmList.append([int(lm.x * w), int(lm.y * h)])
                # Draw landmarks and connections for left hand
                mpDraw.draw_landmarks(img, results.multi_hand_landmarks[0], mpHands.HAND_CONNECTIONS)
            if label == 'Right':
                # Extract and store landmark positions for right hand
                index = 0
                if len(results.multi_hand_landmarks) == 2:
                    index = 1
                for lm in results.multi_hand_landmarks[index].landmark:
                    h, w, _ = img.shape
                    right_lmList.append([int(lm.x * w), int(lm.y * h)])
                    # Draw landmarks and connections for right hand
                    mpDraw.draw_landmarks(img, results.multi_hand_landmarks[index], mpHands.HAND_CONNECTIONS)
   
    # Add a flag to indicate if a gesture is currently being processed
    gesture_in_progress = False

    # Control media playback based on hand gestures
    if left_lmList != []:
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]
        
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        # Left hand gestures
        if not gesture_in_progress:
            if distance_thumb_ring < 65:  # Thumb and ring finger gesture recognized for previous song
                print("Left hand: Thumb and ring finger gesture recognized")
                media_player_thread.previous_song()
                media_player_thread.play()
                gesture_in_progress = True
            elif distance_thumb_middle < 65:  # Thumb and middle finger gesture recognized for next song
                print("Left hand: Thumb and middle finger gesture recognized")
                media_player_thread.next_song()
                media_player_thread.play()
                gesture_in_progress = True
    if right_lmList != []:
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]
        
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        # Right hand gestures
        if not gesture_in_progress:
            if distance_thumb_middle < 65:  # Thumb and middle finger gesture recognized for play/pause
                print("Right hand: Thumb and middle finger gesture recognized")
                if media_player_thread.player.is_playing():
                    media_player_thread.pause()
                else:
                    media_player_thread.play()
                gesture_in_progress = True
            elif distance_thumb_ring < 65:  # Thumb and ring finger gesture recognized for stop
                print("Right hand: Thumb and ring finger gesture recognized")
                media_player_thread.stop()
                gesture_in_progress = True

    # Reset the flag after a short delay to allow for the next gesture
    if gesture_in_progress:
        time.sleep(2.5)  # Adjust the sleep time as needed
        gesture_in_progress = False
                
    # Control screen brightness based on left hand gesture
    if left_lmList != []:
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Control brightness based on thumb and finger gestures
        if distance_thumb_index < 65:
            brightness_levels = sbc.get_brightness()
            brightness = brightness_levels[0] + 10
            sbc.set_brightness(brightness)
            brightness_changed = True
        elif distance_thumb_pinky < 65:
            brightness_levels = sbc.get_brightness()
            brightness = brightness_levels[0] - 10
            sbc.set_brightness(brightness)
            brightness_changed = True

    # Control volume based on right hand gesture
    if right_lmList != []:
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]
        x_pinky, y_pinky = right_lmList[20][0], right_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Control volume based on thumb and finger gestures
        if distance_thumb_index < 65:
            current_vol = volume.GetMasterVolumeLevel()  # Get current volume level
            new_vol = min(current_vol + 0.25, volMax)  # Increase volume by 10 dB, ensuring it does not exceed maximum
            volume.SetMasterVolumeLevel(new_vol, None)
        elif distance_thumb_pinky < 65:
            current_vol = volume.GetMasterVolumeLevel()  # Get current volume level
            new_vol = max(current_vol - 0.25, volMin)  # Decrease volume by 10 dB, ensuring it does not go below minimum
            volume.SetMasterVolumeLevel(new_vol, None)
            

    # Display the annotated image
    cv2.imshow('Image', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xff == ord('q'):
        media_player_thread.stop()
        break

    # Wait for brightness change to stabilize before continuing
    if brightness_changed:
        cv2.waitKey(500)
        brightness_changed = False

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

C:\Users\mzahi\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Right hand: Thumb and middle finger gesture recognized
Right hand: Thumb and middle finger gesture recognized
Right hand: Thumb and ring finger gesture recognized
Right hand: Thumb and middle finger gesture recognized
Right hand: Thumb and middle finger gesture recognized
Right hand: Thumb and middle finger gesture recognized
Left hand: Thumb and middle finger gesture recognized
Left hand: Thumb and middle finger gesture recognized
Left hand: Thumb and middle finger gesture recognized


# Previous Attempts

In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import screen_brightness_control as sbc
import os

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Initialize audio devices
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

# Get volume range
volMin, volMax = volume.GetVolumeRange()[:2]

# Flag to track if brightness changed
brightness_changed = False

# Initialize media files
# media_files = Files(["Khaled - Alech Taadi.mp3", "Major Lazer - Que Calor (feat. J Balvin & El Alfa).mp3","Heilung - In Maidjan.mp3"])
# file1 = File("Khaled - Alech Taadi.mp3")
# file2 = File("Major Lazer - Que Calor (feat. J Balvin & El Alfa).mp3")
# file3 = File("Heilung - In Maidjan.mp3")

# Main loop to capture frames from webcam
while True:
    # Capture frame from webcam
    success, img = cap.read()
    
    # Flip the frame horizontally for natural hand movements
    img = cv2.flip(img, 1)
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList, right_lmList = [], []
    
    # Check if hands are detected and determine their labels (left or right)
    if results.multi_hand_landmarks and results.multi_handedness:
        for i in results.multi_handedness:
            label = i.classification[0].label
            if label == 'Left':
                # Extract and store landmark positions for left hand
                for lm in results.multi_hand_landmarks[0].landmark:
                    h, w, _ = img.shape
                    left_lmList.append([int(lm.x * w), int(lm.y * h)])
                # Draw landmarks and connections for left hand
                mpDraw.draw_landmarks(img, results.multi_hand_landmarks[0], mpHands.HAND_CONNECTIONS)
            if label == 'Right':
                # Extract and store landmark positions for right hand
                index = 0
                if len(results.multi_hand_landmarks) == 2:
                    index = 1
                for lm in results.multi_hand_landmarks[index].landmark:
                    h, w, _ = img.shape
                    right_lmList.append([int(lm.x * w), int(lm.y * h)])
                    # Draw landmarks and connections for right hand
                    mpDraw.draw_landmarks(img, results.multi_hand_landmarks[index], mpHands.HAND_CONNECTIONS)

    # Control screen brightness based on left hand gesture
    if left_lmList != []:
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Control brightness based on thumb and finger gestures
        if distance_thumb_index < 65:
            brightness_levels = sbc.get_brightness()
            brightness = brightness_levels[0] + 10
            sbc.set_brightness(brightness)
            brightness_changed = True
        elif distance_thumb_pinky < 65:
            brightness_levels = sbc.get_brightness()
            brightness = brightness_levels[0] - 10
            sbc.set_brightness(brightness)
            brightness_changed = True


    # Control volume based on right hand gesture
    if right_lmList != []:
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]
        x_pinky, y_pinky = right_lmList[20][0], right_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Control volume based on thumb and finger gestures
        if distance_thumb_index < 65:
            current_vol = volume.GetMasterVolumeLevel()  # Get current volume level
            new_vol = min(current_vol + 0.25, volMax)  # Increase volume by 10 dB, ensuring it does not exceed maximum
            volume.SetMasterVolumeLevel(new_vol, None)
        elif distance_thumb_pinky < 65:
            current_vol = volume.GetMasterVolumeLevel()  # Get current volume level
            new_vol = max(current_vol - 0.25, volMin)  # Decrease volume by 10 dB, ensuring it does not go below minimum
            volume.SetMasterVolumeLevel(new_vol, None)

    # Control media playback based on hand gestures
    if len(left_lmList) >= 21 and len(right_lmList) >= 21:  # Ensure all landmarks are detected for both hands
        # Left hand ring finger for previous
        x_ring_left, y_ring_left = left_lmList[16][0], left_lmList[16][1]  # Ring finger landmark for left hand
        if distance_thumb_ring_left < THRESHOLD:  # Adjust THRESHOLD according to your preference
            print("Left hand: Thumb and ring finger gesture recognized")
            win32api.keybd_event(0xB1, 0, 0, 0)  # 0xB1 is the VK_MEDIA_PREV_TRACK key code

        # Left hand middle finger for next
        x_middle_left, y_middle_left = left_lmList[12][0], left_lmList[12][1]  # Middle finger landmark for left hand
        if distance_thumb_middle_left < THRESHOLD:  # Adjust THRESHOLD according to your preference
            print("Left hand: Thumb and middle finger gesture recognized")
            win32api.keybd_event(0xB0, 0, 0, 0)  # 0xB0 is the VK_MEDIA_NEXT_TRACK key code

        # Right hand middle finger for play
        x_middle_right, y_middle_right = right_lmList[12][0], right_lmList[12][1]  # Middle finger landmark for right hand
        if distance_thumb_middle_right < THRESHOLD:  # Adjust THRESHOLD according to your preference
            print("Right hand: Thumb and middle finger gesture recognized")
            win32api.keybd_event(0xB3, 0, 0, 0)  # 0xB3 is the VK_MEDIA_PLAY_PAUSE key code

        # Right hand ring finger for pause
        x_ring_right, y_ring_right = right_lmList[16][0], right_lmList[16][1]  # Ring finger landmark for right hand
        if distance_thumb_ring_right < THRESHOLD:  # Adjust THRESHOLD according to your preference
            print("Right hand: Thumb and ring finger gesture recognized")
            win32api.keybd_event(0xB3, 0, 0, 0)  # 0xB3 is the VK_MEDIA_PLAY_PAUSE key code

    # Display the annotated image
    cv2.imshow('Image', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

    # Wait for brightness change to stabilize before continuing
    if brightness_changed:
        cv2.waitKey(500)
        brightness_changed = False

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()


In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import screen_brightness_control as sbc

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Initialize audio devices
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

# Get volume range
volMin, volMax = volume.GetVolumeRange()[:2]

# Flag to track if brightness changed
brightness_changed = False

# Main loop to capture frames from webcam
while True:
    # Capture frame from webcam
    success, img = cap.read()
    
    # Flip the frame horizontally for natural hand movements
    img = cv2.flip(img, 1)
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList, right_lmList = [], []
    
    # Check if hands are detected and determine their labels (left or right)
    if results.multi_hand_landmarks and results.multi_handedness:
        for i in results.multi_handedness:
            label = i.classification[0].label
            if label == 'Left':
                # Extract and store landmark positions for left hand
                for lm in results.multi_hand_landmarks[0].landmark:
                    h, w, _ = img.shape
                    left_lmList.append([int(lm.x * w), int(lm.y * h)])
                # Draw landmarks and connections for left hand
                mpDraw.draw_landmarks(img, results.multi_hand_landmarks[0], mpHands.HAND_CONNECTIONS)
            if label == 'Right':
                # Extract and store landmark positions for right hand
                index = 0
                if len(results.multi_hand_landmarks) == 2:
                    index = 1
                for lm in results.multi_hand_landmarks[index].landmark:
                    h, w, _ = img.shape
                    right_lmList.append([int(lm.x * w), int(lm.y * h)])
                    # Draw landmarks and connections for right hand
                    mpDraw.draw_landmarks(img, results.multi_hand_landmarks[index], mpHands.HAND_CONNECTIONS)

    # Control screen brightness based on left hand gesture
    if left_lmList != []:
        x1, y1 = left_lmList[4][0], left_lmList[4][1]
        x2, y2 = left_lmList[8][0], left_lmList[8][1]

        # Calculate length of line formed by thumb and index finger
        length = hypot(x2 - x1, y2 - y1)

        # Interpolate brightness based on length of line
        bright = np.interp(length, [50, 200], [0, 100])
        
        # Check if there's a significant difference before changing brightness
        if abs(bright - sbc.get_brightness()) > 5:
            sbc.set_brightness(int(bright))
            brightness_changed = True

    # Control volume based on right hand gesture
    if right_lmList != []:
        x1, y1 = right_lmList[4][0], right_lmList[4][1]
        x2, y2 = right_lmList[8][0], right_lmList[8][1]

        # Calculate length of line formed by thumb and index finger
        length = hypot(x2 - x1, y2 - y1)

        # Interpolate volume based on length of line
        vol = np.interp(length, [50, 200], [volMin, volMax])
        volume.SetMasterVolumeLevel(vol, None)

    # Recognize other finger combinations for left hand
    if left_lmList != []:
        # Extract x and y coordinates of each finger landmark
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Print recognized gestures without assigning actions
        if distance_thumb_index < 65:
            print("Left hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < 65:
            print("Left hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < 65:
            print("Left hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < 65:
            print("Left hand: Thumb and pinky finger gesture recognized")

    # Recognize other finger combinations for right hand
    if right_lmList != []:
        # Extract x and y coordinates of each finger landmark
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]
        x_pinky, y_pinky = right_lmList[20][0], right_lmList[20][1]

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Print recognized gestures without assigning actions
        if distance_thumb_index < 65:
            print("Right hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < 65:
            print("Right hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < 65:
            print("Right hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < 65:
            print("Right hand: Thumb and pinky finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Image', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

    # Wait for brightness change to stabilize before continuing
    if brightness_changed:
        cv2.waitKey(500)
        brightness_changed = False

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import screen_brightness_control as sbc

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Initialize audio devices
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

# Get volume range
volMin, volMax = volume.GetVolumeRange()[:2]

# Flag to track if brightness changed
brightness_changed = False

# Main loop to capture frames from webcam
while True:
    # Capture frame from webcam
    success, img = cap.read()
    
    # Flip the frame horizontally for natural hand movements
    img = cv2.flip(img, 1)
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList, right_lmList = [], []
    
    # Check if hands are detected and determine their labels (left or right)
    if results.multi_hand_landmarks and results.multi_handedness:
        for i in results.multi_handedness:
            label = i.classification[0].label
            if label == 'Left':
                # Extract and store landmark positions for left hand
                for lm in results.multi_hand_landmarks[0].landmark:
                    h, w, _ = img.shape
                    left_lmList.append([int(lm.x * w), int(lm.y * h)])
                # Draw landmarks and connections for left hand
                mpDraw.draw_landmarks(img, results.multi_hand_landmarks[0], mpHands.HAND_CONNECTIONS)
            if label == 'Right':
                # Extract and store landmark positions for right hand
                index = 0
                if len(results.multi_hand_landmarks) == 2:
                    index = 1
                for lm in results.multi_hand_landmarks[index].landmark:
                    h, w, _ = img.shape
                    right_lmList.append([int(lm.x * w), int(lm.y * h)])
                    # Draw landmarks and connections for right hand
                    mpDraw.draw_landmarks(img, results.multi_hand_landmarks[index], mpHands.HAND_CONNECTIONS)

    # Control screen brightness based on left hand gesture
    if left_lmList != []:
        x1, y1 = left_lmList[4][0], left_lmList[4][1]
        x2, y2 = left_lmList[8][0], left_lmList[8][1]

        # Calculate length of line formed by thumb and index finger
        length = hypot(x2 - x1, y2 - y1)

        # Interpolate brightness based on length of line
        bright = np.interp(length, [50, 200], [0, 100])
        
        # Check if there's a significant difference before changing brightness
        if abs(bright - sbc.get_brightness()) > 5:
            sbc.set_brightness(int(bright))
            brightness_changed = True

    # Control volume based on right hand gesture
    if right_lmList != []:
        x1, y1 = right_lmList[4][0], right_lmList[4][1]
        x2, y2 = right_lmList[8][0], right_lmList[8][1]

        # Calculate length of line formed by thumb and index finger
        length = hypot(x2 - x1, y2 - y1)

        # Interpolate volume based on length of line
        vol = np.interp(length, [50, 200], [volMin, volMax])
        volume.SetMasterVolumeLevel(vol, None)

    # Display the annotated image
    cv2.imshow('Image', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

    # Wait for brightness change to stabilize before continuing
    if brightness_changed:
        cv2.waitKey(500)
        brightness_changed = False

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()


In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Threshold for finger distances
THRESHOLD = 50

# Main loop for capturing and processing frames
while True:
    # Capture frame from webcam
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip horizontally for natural hand movements
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # List to store landmark positions of left hand
    left_lmList = []

    # Check if left hand landmarks are detected
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            for lm in handLandmarks.landmark:
                # Store landmark positions of left hand
                h, w, _ = img.shape
                left_lmList.append([int(lm.x * w), int(lm.y * h)])
            # Draw landmarks and connections for left hand
            mpDraw.draw_landmarks(img, handLandmarks, mpHands.HAND_CONNECTIONS)

    # Recognize thumb and finger gestures in the left hand
    if len(left_lmList) >= 21:  # Ensure all landmarks are detected
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]  # Thumb landmark
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]  # Index finger landmark
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]  # Middle finger landmark
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]  # Ring finger landmark
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]  # Pinky finger landmark

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Define thresholds for distances to recognize different finger gestures
        if distance_thumb_index < THRESHOLD:
            print("Thumb and index finger gesture recognized")
        elif distance_thumb_middle < THRESHOLD:
            print("Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < THRESHOLD:
            print("Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < THRESHOLD:
            print("Thumb and pinky finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Hand Gesture Detection', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75, max_num_hands=2)  # Set max_num_hands to 2
mpDraw = mp.solutions.drawing_utils

# Threshold for finger distances
THRESHOLD = 100

# Main loop for capturing and processing frames
while True:
    # Capture frame from webcam
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip horizontally for natural hand movements
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList = []
    right_lmList = []

    # Check if hand landmarks are detected
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            # Extract hand landmarks and draw landmarks and connections
            mpDraw.draw_landmarks(img, handLandmarks, mpHands.HAND_CONNECTIONS)

            # Store landmark positions of left and right hands
            for idx, lm in enumerate(handLandmarks.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)  # Landmark coordinates
                if idx == 0:  # Check landmark index to determine left or right hand
                    left_lmList.append([cx, cy])
                else:
                    right_lmList.append([cx, cy])

    # Recognize thumb and finger gestures for left hand
    if len(left_lmList) >= 21:  # Ensure all landmarks are detected
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]  # Thumb landmark
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]  # Index finger landmark
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]  # Middle finger landmark
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]  # Ring finger landmark
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]  # Pinky finger landmark

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Define thresholds for distances to recognize different finger gestures
        if distance_thumb_index < THRESHOLD:
            print("Left hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < THRESHOLD:
            print("Left hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < THRESHOLD:
            print("Left hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < THRESHOLD:
            print("Left hand: Thumb and pinky finger gesture recognized")

    # Recognize thumb and finger gestures for right hand
    if len(right_lmList) >= 21:  # Ensure all landmarks are detected
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]  # Thumb landmark
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]  # Index finger landmark
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]  # Middle finger landmark
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]  # Ring finger landmark
        x_pinky, y_pinky = right_lmList[20][0], right_lmList[20][1]  # Pinky finger landmark

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Define thresholds for distances to recognize different finger gestures
        if distance_thumb_index < THRESHOLD:
            print("Right hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < THRESHOLD:
            print("Right hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < THRESHOLD:
            print("Right hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < THRESHOLD:
            print("Right hand: Thumb and pinky finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Hand Gesture Detection', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75, max_num_hands=2)  # Set max_num_hands to 2
mpDraw = mp.solutions.drawing_utils

# Threshold for finger distances
THRESHOLD = 65  # Adjust this threshold value

# Main loop for capturing and processing frames
while True:
    # Capture frame from webcam
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip horizontally for natural hand movements
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList = []
    right_lmList = []

    # Check if hand landmarks are detected
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            # Extract hand landmarks and draw landmarks and connections
            mpDraw.draw_landmarks(img, handLandmarks, mpHands.HAND_CONNECTIONS)

            # Store landmark positions of left and right hands
            for idx, lm in enumerate(handLandmarks.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)  # Landmark coordinates
                if cx < w // 2:  # Check if landmark is on the left or right side of the frame
                    left_lmList.append([cx, cy])
                else:
                    right_lmList.append([cx, cy])

    # Recognize thumb and finger gestures for left hand
    if len(left_lmList) >= 21:  # Ensure all landmarks are detected for left hand
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]  # Thumb landmark
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]  # Index finger landmark
        x_middle, y_middle = left_lmList[12][0], left_lmList[12][1]  # Middle finger landmark
        x_ring, y_ring = left_lmList[16][0], left_lmList[16][1]  # Ring finger landmark
        x_pinky, y_pinky = left_lmList[20][0], left_lmList[20][1]  # Pinky finger landmark

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Define thresholds for distances to recognize different finger gestures
        if distance_thumb_index < THRESHOLD:
            print("Left hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < THRESHOLD:
            print("Left hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < THRESHOLD:
            print("Left hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < THRESHOLD:
            print("Left hand: Thumb and pinky finger gesture recognized")

    # Recognize thumb and finger gestures for right hand
    if len(right_lmList) >= 21:  # Ensure all landmarks are detected for right hand
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]  # Thumb landmark
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]  # Index finger landmark
        x_middle, y_middle = right_lmList[12][0], right_lmList[12][1]  # Middle finger landmark
        x_ring, y_ring = right_lmList[16][0], right_lmList[16][1]  # Ring finger landmark
        x_pinky, y_pinky = right_lmList[20][0], right_lmList[20][1]  # Pinky finger landmark

        # Calculate distances between thumb and other fingers
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))
        distance_thumb_middle = calculate_distance((x_thumb, y_thumb), (x_middle, y_middle))
        distance_thumb_ring = calculate_distance((x_thumb, y_thumb), (x_ring, y_ring))
        distance_thumb_pinky = calculate_distance((x_thumb, y_thumb), (x_pinky, y_pinky))

        # Define thresholds for distances to recognize different finger gestures
        if distance_thumb_index < THRESHOLD:
            print("Right hand: Thumb and index finger gesture recognized")
        elif distance_thumb_middle < THRESHOLD:
            print("Right hand: Thumb and middle finger gesture recognized")
        elif distance_thumb_ring < THRESHOLD:
            print("Right hand: Thumb and ring finger gesture recognized")
        elif distance_thumb_pinky < THRESHOLD:
            print("Right hand: Thumb and pinky finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Hand Gesture Detection', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75)
mpDraw = mp.solutions.drawing_utils

# Threshold for thumb and middle finger distance
THRESHOLD = 50

# Main loop for capturing and processing frames
while True:
    # Capture frame from webcam
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip horizontally for natural hand movements
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # List to store landmark positions of left hand
    left_lmList = []

    # Check if left hand landmarks are detected
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            for lm in handLandmarks.landmark:
                # Store landmark positions of left hand
                h, w, _ = img.shape
                left_lmList.append([int(lm.x * w), int(lm.y * h)])
            # Draw landmarks and connections for left hand
            mpDraw.draw_landmarks(img, handLandmarks, mpHands.HAND_CONNECTIONS)

    # Recognize thumb and middle finger gesture in the left hand
    if len(left_lmList) >= 9:  # Ensure all landmarks are detected
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]  # Thumb landmark
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]  # Index finger landmark

        # Calculate Euclidean distance between thumb and index finger landmarks
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))

        # If the distance is below the threshold, recognize the gesture
        if distance_thumb_index < THRESHOLD:
            print("Thumb and index finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Hand Gesture Detection', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import mediapipe as mp
from math import hypot

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return hypot(point2[0] - point1[0], point2[1] - point1[1])

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands module
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.75, max_num_hands=2)  # Set max_num_hands to 2
mpDraw = mp.solutions.drawing_utils

# Threshold for finger distances
THRESHOLD = 50

# Main loop for capturing and processing frames
while True:
    # Capture frame from webcam
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip horizontally for natural hand movements
    
    # Convert frame to RGB format for processing by MediaPipe
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    results = hands.process(imgRGB)

    # Lists to store landmark positions of left and right hands
    left_lmList = []
    right_lmList = []

    # Check if hand landmarks are detected
    if results.multi_hand_landmarks:
        for handLandmarks in results.multi_hand_landmarks:
            # Extract hand landmarks and draw landmarks and connections
            mpDraw.draw_landmarks(img, handLandmarks, mpHands.HAND_CONNECTIONS)

            # Store landmark positions of left and right hands
            for idx, lm in enumerate(handLandmarks.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)  # Landmark coordinates
                if cx < w // 2:  # Check if landmark is on the left or right side of the frame
                    left_lmList.append([cx, cy])
                else:
                    right_lmList.append([cx, cy])

    # Recognize thumb and finger gestures for left hand
    if len(left_lmList) >= 21:  # Ensure all landmarks are detected for left hand
        x_thumb, y_thumb = left_lmList[4][0], left_lmList[4][1]  # Thumb landmark
        x_index, y_index = left_lmList[8][0], left_lmList[8][1]  # Index finger landmark

        # Calculate Euclidean distance between thumb and index finger landmarks
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))

        # If the distance is below the threshold, recognize the gesture
        if distance_thumb_index < THRESHOLD:
            print("Left hand: Thumb and index finger gesture recognized")

    # Recognize thumb and finger gestures for right hand
    if len(right_lmList) >= 21:  # Ensure all landmarks are detected for right hand
        x_thumb, y_thumb = right_lmList[4][0], right_lmList[4][1]  # Thumb landmark
        x_index, y_index = right_lmList[8][0], right_lmList[8][1]  # Index finger landmark

        # Calculate Euclidean distance between thumb and index finger landmarks
        distance_thumb_index = calculate_distance((x_thumb, y_thumb), (x_index, y_index))

        # If the distance is below the threshold, recognize the gesture
        if distance_thumb_index < THRESHOLD:
            print("Right hand: Thumb and index finger gesture recognized")

    # Display the annotated image
    cv2.imshow('Hand Gesture Detection', img)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cv2.destroyAllWindows()
cap.release()



In [ ]:
import vlc 
import os
import time
path = "media/"
os.listdir(path)

In [ ]:
L=os.listdir(path)
song = vlc.MediaPlayer(L[0])
song.play()
print('is_playing:', song.is_playing())  # 0 = False

time.sleep(5)  # sleep because it needs time to start playing

print('is_playing:', song.is_playing())  # 1 = True

while song.is_playing():
    time.sleep(5)  # sleep to use less CPU

In [ ]:
import vlc 
import os
import time
import threading

path = "media/"
os.listdir(path)

class MediaPlayerThread(threading.Thread):
    def __init__(self, file_path):
        super().__init__()
        self.song = vlc.MediaPlayer(file_path)
    
    def run(self):
        self.song.play()
        while True:
            time.sleep(1)

L = os.listdir(path)
media_player_thread = MediaPlayerThread(os.path.join(path, L[0]))
media_player_thread.start()

# Adding a short delay to allow the media player to initialize
time.sleep(1)

print('is_playing:', media_player_thread.song.is_playing())

# Now you can pause the song from another cell



In [ ]:
media_player_thread.song.pause()

In [ ]:
media_player_thread.song.play()

In [ ]:
media_player_thread.song.pause()

In [ ]:
media_player_thread.song.play()

In [ ]:
media_player_thread.song.stop()

In [ ]:
media_player_thread.song.play()

In [ ]:
media_player_thread.song.stop()

In [ ]:
import vlc 
import os
import time
import threading

class MediaPlayerThread(threading.Thread):
    def __init__(self, file_paths):
        super().__init__()
        self.file_paths = file_paths
        self.current_index = 0
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
    
    def play(self):
        self.player.play()
    
    def pause(self):
        self.player.pause()
        
    def stop(self):
        self.player.stop()
    
    def next_song(self):
        self.current_index = (self.current_index + 1) % len(self.file_paths)
        self.player.stop()
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
        self.player.play()
    
    def previous_song(self):
        self.current_index = (self.current_index - 1) % len(self.file_paths)
        self.player.stop()
        self.player = vlc.MediaPlayer(self.file_paths[self.current_index])
        self.player.play()
    
    def run(self):
        self.play()
        while True:
            time.sleep(1)  # Adjust the sleep time as needed

path = "media/"
file_names = os.listdir(path)
file_paths = [os.path.join(path, file) for file in file_names]

media_player_thread = MediaPlayerThread(file_paths)
media_player_thread.start()

# Adding a short delay to allow the media player to initialize
time.sleep(1)

print('is_playing:', media_player_thread.player.is_playing())

# Now you can control playback from other cells


In [ ]:
media_player_thread.next_song()

In [ ]:
media_player_thread.previous_song()

In [ ]:
media_player_thread.pause()

In [ ]:
media_player_thread.play()

In [ ]:
media_player_thread.stop()

In [ ]:
media_player_thread.play()

In [ ]:
media_player_thread.stop()